In [2]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import joblib

In [ ]:
#Queue
queue_list = pd.read_excel('D:/Code/Code/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers

#Variable
Read_Date_variable = pd.ExcelFile('D:/Code/Code/Support_dashboard.xlsx')
Date_variable = pd.read_excel(Read_Date_variable, sheet_name='Variable', dtype={'Start date': object, 'End date': object})
Date_variable

,Date to Export,Fields,Start date,End date
0,Appeal Daily Report,Diff case,2023-07-19,2023-07-19
1,Appeal Daily Report,Daily Report,2023-07-18,2023-07-18
2,Regular,Diff case,2023-07-17,2023-07-19
3,Regular,Daily Report,2023-07-18,2023-07-18
4,People statistic,Daily Report,2023-07-18,2023-07-18
5,Product Comment Report,Diff case,2023-07-19,2023-07-19
6,Product Comment Report,Daily Report,2023-07-18,2023-07-18
7,Export Gov Oec,Diff case,2023-07-19,2023-07-19
8,Export Gov Oec,Diff case,2023-07-19,2023-07-19
9,UPL,Diff case,NaT,NaT


In [ ]:
Start_Date_Gov = datetime.strptime(Date_variable.loc[7, 'Start date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Gov = datetime.strptime(Date_variable.loc[7, 'End date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_Oec = datetime.strptime(Date_variable.loc[8,'Start date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Oec = datetime.strptime(Date_variable.loc[8,'End date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_gov_oec= datetime.strptime(Date_variable.loc[14,'Start date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_gov_oec = datetime.strptime(Date_variable.loc[14,'End date'].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime('%Y-%m-%d')

In [ ]:
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)    
def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.csv'):
        list_files.append(filename)

    df1 = pd.concat(map(pd.read_csv, list_files), ignore_index=True)
    
    return df1


In [ ]:
Gov_Rawdata = input_data('D:/Gov and OEC/Gov')
OEC_Rawdata = input_data('D:/Gov and OEC/OEC')

In [ ]:
def filter_time( df,  from_date, to_date):
    Gov_Rawdata['1_resolve_time'] = pd.to_datetime(Gov_Rawdata['1_resolve_time'])
    gov_list_files_final = df[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]

    return gov_list_files_final

Gov_Rawdata = filter_time(Gov_Rawdata, Start_Date_Gov,End_Date_Gov)

In [ ]:
def filter_time( df,  from_date, to_date):
    OEC_Rawdata['1_resolve_time'] = pd.to_datetime(OEC_Rawdata['1_resolve_time'])
    gov_list_files_final = df[(df['1_resolve_time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['1_resolve_time'].dt.strftime('%Y-%m-%d') <= to_date)]

    return gov_list_files_final

OEC_Rawdata = filter_time(OEC_Rawdata,Start_Date_Oec,End_Date_Oec)

In [ ]:
verifiers = Gov_Rawdata['1_verifier'].unique()
gov_list_files = []

try:
    for v in verifiers:
        sample_frac = Gov_Rawdata[Gov_Rawdata['1_verifier'] == v].sample(frac=0.03)
        gov_list_files.append(sample_frac)
    if len(gov_list_files) > 0:
        gov_list_files = pd.concat(gov_list_files, ignore_index=True)
        gov_list_files['project_id']= gov_list_files['project_id'].astype(str).str.replace('id=','')
        gov_list_files['task_id']= gov_list_files['task_id'].astype(str).str.replace('id=','')
        gov_list_files['object_id']= gov_list_files['object_id'].astype(str).str.replace('id=','')
        gov_list_files = gov_list_files[['title','project_id','1_verifier','1_resolve_time','object_id','1_audit_reject','1_reason','1_duration','task_id','tcs_link']]
except:
    pass

gov_list_files_final = pd.DataFrame(gov_list_files)
gov_list_files_final.columns

Index(['title', 'project_id', '1_verifier', '1_resolve_time', 'object_id',
       '1_audit_reject', '1_reason', '1_duration', 'task_id', 'tcs_link'],
      dtype='object')

In [ ]:
verifiers = OEC_Rawdata['1_verifier'].unique()
oec_list_files = []

try:
   for v in verifiers:
      sample_frac = OEC_Rawdata[OEC_Rawdata['1_verifier'] == v].sample(frac=0.1)
      oec_list_files.append(sample_frac)
   if len(oec_list_files) > 0:
      oec_list_files = pd.concat(oec_list_files, ignore_index=True)
      oec_list_files['project_id']= oec_list_files['project_id'].astype(str).str.replace('id=','')
      oec_list_files['task_id']= oec_list_files['task_id'].astype(str).str.replace('id=','')
      oec_list_files['object_id']= oec_list_files['object_id'].astype(str).str.replace('id=','')
      oec_list_files = oec_list_files[['title','project_id','1_verifier','1_resolve_time','object_id','1_audit_result','1_reject_label','1_duration','task_id','tcs_link']].rename(columns={'1_audit_result':'1_audit_reject','1_reject_label':'1_reason'})
except:
   pass
oec_list_files_final = pd.DataFrame(oec_list_files)
oec_list_files_final.columns

Index(['title', 'project_id', '1_verifier', '1_resolve_time', 'object_id',
       '1_audit_reject', '1_reason', '1_duration', 'task_id', 'tcs_link'],
      dtype='object')

In [ ]:
if len(gov_list_files_final) > 0 and len(oec_list_files_final) > 0:
    gov_and_oec_Queue_temp = pd.concat([gov_list_files_final,oec_list_files_final],axis=0)
elif len(gov_list_files_final) > 0:
    gov_and_oec_Queue_temp = gov_list_files_final
elif len(oec_list_files_final) > 0:
    gov_and_oec_Queue_temp = oec_list_files_final
else:
    gov_and_oec_Queue_temp = None

In [ ]:
gov_and_oec_Queue_temp.head(5)

,title,project_id,1_verifier,1_resolve_time,object_id,1_audit_reject,1_reason,1_duration,task_id,tcs_link
0,[VN] Seller Gov X Feige Chat,7202624223037227521,ngoc.nhm1@trans-cosmos.com.vn,2023-07-15 11:22:10,recheck_7255312260493574406_1689389007,Reject,Redirecting purchase outside platform,106,7255872072952431110,https://tcs-va.bytelemon.com/worktable/7202624...
1,[VN] Seller Gov X Feige Chat,7202624223037227521,khoa.nd@trans-cosmos.com.vn,2023-07-15 12:21:43,recheck_7255608394613129478_1689389243,Reject,Offering off-platform return refund to buyers,331,7255872079742943750,https://tcs-va.bytelemon.com/worktable/7202624...
2,[VN] Seller Gov X Feige Chat,7202624223037227521,khoa.nd@trans-cosmos.com.vn,2023-07-15 10:20:46,recheck_7255640308569899269_1689388526,Reject,Redirecting purchase outside platform,103,7255872065226490373,https://tcs-va.bytelemon.com/worktable/7202624...
3,[VN] Seller Gov X Feige Chat,7202624223037227521,uyen.ntt4@trans-cosmos.com.vn,2023-07-15 10:07:08,recheck_7255324205582303494_1689388391,Reject,Redirecting purchase outside platform,24,7255872070087705094,https://tcs-va.bytelemon.com/worktable/7202624...
4,[VN] Seller Gov X Feige Chat,7202624223037227521,thach.vt@trans-cosmos.com.vn,2023-07-15 12:18:02,recheck_7255592077889716485_1689389230,Reject,Redirecting purchase outside platform,634,7255872077477986821,https://tcs-va.bytelemon.com/worktable/7202624...


In [ ]:
keymap = {
    'Not approve': ['400', 'notapproval', 'disapproval', 'disapproved', 'violation','reject'],
    'Approve': ['200', 'approval', 'approved', 'general'],
}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()

        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status.capitalize()
    return s
gov_and_oec_Queue_temp['1_audit_reject'] = gov_and_oec_Queue_temp['1_audit_reject'].apply(check_string)

In [ ]:
dir_path = 'D:/Data clean/OEC and Seller GOV chat'

dfs = []
for file in os.listdir(dir_path):
    if file.endswith('.xlsx'):
        file_path = os.path.join(dir_path, file)
        df = pd.read_excel(file_path, engine='openpyxl',dtype={'Queue ID':object,'object id':object,'BPO Task ID':object})
        dfs.append(df)

OEC_GOV_df = pd.concat(dfs, ignore_index=True)
# OEC_GOV_df['mod_date'] = pd.to_datetime(OEC_GOV_df['mod_date'].apply(convert_date))
OEC_GOV_df['mod_date']

0       2023-06-01 18:49:21
1       2023-06-01 18:50:22
2       2023-06-01 19:27:44
3       2023-06-01 21:28:59
4       2023-06-01 16:51:28
               ...         
6087    2023-07-15 10:36:38
6088    2023-07-15 10:53:19
6089    2023-07-15 12:25:57
6090    2023-07-14 10:33:13
6091    2023-07-15 12:21:05
Name: mod_date, Length: 6092, dtype: object

In [ ]:
# def filter_time( df,  from_date, to_date):
#     OEC_GOV_df['mod_date'] = pd.to_datetime(OEC_GOV_df['mod_date'])
#     OEC_GOV_df_final = df[(df['mod_date'].dt.strftime('%Y-%m-%d') >= from_date) & (df['mod_date'].dt.strftime('%Y-%m-%d') <= to_date)]

#     return OEC_GOV_df_final

# OEC_GOV_df = filter_time(OEC_GOV_df, Start_Date_gov_oec,End_Date_gov_oec)
# OEC_GOV_df

In [ ]:
OEC_GOV_df.columns

Index(['Queue Name', 'Queue ID', 'mod', 'mod_date', 'object id', 'mod_result',
       'mod_reason', 'mod_aht', 'BPO Task ID', 'BPO Link', 'BPO QA Result',
       'BPO QA Reason', 'Diff_Case', 'BPO QA Task ID', 'BPO QA Link',
       'BPO QA Note', 'Diff Case'],
      dtype='object')

In [ ]:
OEC_GOV_df['mod_date'] = pd.to_datetime(OEC_GOV_df['mod_date'].apply(convert_date))

In [ ]:
#Sample size
sample_size_filter = OEC_GOV_df[['mod_date','Queue Name','mod','object id']].rename(columns={'mod_date':'Moderation time',
                                                                                           'Queue Name':'Sampling Queue',
                                                                                           'mod':'Moderator name',
                                                                                           'object id':'No. of Samples'})
def filter_time( df, from_date, to_date):

    sample_size_filter = df[(df['Moderation time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['Moderation time'].dt.strftime('%Y-%m-%d') <= to_date)]
    sample_size_filter = sample_size_filter.groupby(by=['Moderation time','Sampling Queue','Moderator name'], as_index=False).agg({'No. of Samples':'count'})

    sample_size_filter = sample_size_filter[['Moderation time','Sampling Queue','Moderator name','No. of Samples']]

    return sample_size_filter, from_date, to_date

# Fill in moderation time
sample_size_filter,from_date, to_date = filter_time(sample_size_filter, Start_Date_gov_oec,End_Date_gov_oec)
sample_size_filter

,Moderation time,Sampling Queue,Moderator name,No. of Samples
0,2023-07-10,[VN] OEC Product Model Recall Review,duyen.pmk@trans-cosmos.com.vn,4
1,2023-07-10,[VN] OEC Product Model Recall Review,hanh.nt2@trans-cosmos.com.vn,9
2,2023-07-10,[VN] OEC Product Model Recall Review,thao.ttt@trans-cosmos.com.vn,11
3,2023-07-10,[VN] OEC Product Model Recall Review,thien.th@trans-cosmos.com.vn,4
4,2023-07-11,[VN] OEC Product Model Recall Review,duyen.pmk@trans-cosmos.com.vn,3
5,2023-07-11,[VN] OEC Product Model Recall Review,hanh.nt2@trans-cosmos.com.vn,2
6,2023-07-11,[VN] OEC Product Model Recall Review,hieu.ntm@trans-cosmos.com.vn,2
7,2023-07-11,[VN] OEC Product Model Recall Review,thao.ttt@trans-cosmos.com.vn,4
8,2023-07-11,[VN] Seller Gov X Feige Chat,an.lht@trans-cosmos.com.vn,1
9,2023-07-11,[VN] Seller Gov X Feige Chat,huong.vtl@trans-cosmos.com.vn,1


In [ ]:
OEC_GOV_df['mod_result'].unique()

array(['Approval', 'Disapproval', 'Approve', 'Reject', 'Unsure',
       'Not approve'], dtype=object)

In [ ]:
keymap = {
    'Not Approve': ['disapproval','reject', 'unsure','not approve'],
    'Approve': ['approval', 'approve'],
}

def check_string(s):
    if isinstance(s, str):
        s_lower = s.lower()
    
        for approval_status, substrings in keymap.items():
            if any(substring in s_lower for substring in substrings):
                return approval_status.capitalize()

    return s

OEC_GOV_df['mod_result'] = OEC_GOV_df['mod_result'].apply(check_string)
OEC_GOV_df['BPO QA Result'] = OEC_GOV_df['BPO QA Result'].apply(check_string)


In [ ]:
#Add diff case
OEC_GOV_df.loc[OEC_GOV_df['mod_result'] != OEC_GOV_df['BPO QA Result'],'Diff Case']="FALSE"
OEC_GOV_df.loc[OEC_GOV_df['Diff Case'].isnull(),'Diff Case']="TRUE"

In [ ]:
#Diff case
diff_case_filter = OEC_GOV_df[['mod_date','Queue Name','mod','Diff Case']].rename(columns={'mod_date':'Moderation time',
                                                                                           'Queue Name':'Sampling Queue',
                                                                                           'mod':'Moderator name',
                                                                                           'Diff Case':'No. Diff cases'})
diff_case_filter = diff_case_filter[diff_case_filter['No. Diff cases']=="FALSE"]

In [ ]:
#Diff case
false_case_filter = OEC_GOV_df[['mod_date','Queue Name','mod','Diff Case']].rename(columns={'mod_date':'Moderation time',
                                                                                           'Queue Name':'Sampling Queue',
                                                                                           'mod':'Moderator name',
                                                                                           'Diff Case':'No. Diff cases'})
false_case_filter = false_case_filter[false_case_filter['No. Diff cases']=="FALSE"]

def filter_time( df, from_date, to_date):

    false_case_filter = df[(df['Moderation time'].dt.strftime('%Y-%m-%d') >= from_date) & (df['Moderation time'].dt.strftime('%Y-%m-%d') <= to_date)]
    false_case_filter = false_case_filter.groupby(by=['Moderation time','Sampling Queue','Moderator name'], as_index=False).agg({'No. Diff cases':'count'})

    false_case_filter = false_case_filter[['Moderation time','Sampling Queue','Moderator name','No. Diff cases']]

    return false_case_filter, from_date, to_date

# Fill in moderation time
false_case_filter,from_date, to_date = filter_time(false_case_filter,Start_Date_gov_oec,End_Date_gov_oec)

In [ ]:
#Daily report
daily_performance = pd.merge(sample_size_filter,false_case_filter,how='left',left_on=['Moderation time','Sampling Queue','Moderator name'],right_on=['Moderation time','Sampling Queue','Moderator name'])
daily_performance = daily_performance.drop_duplicates(subset=['Moderation time', 'Sampling Queue', 'Moderator name'],keep='first')
daily_performance['No. Diff cases'].fillna(0, inplace=True)

daily_performance[['Moderation time','Sampling Queue','Moderator name','No. of Samples','No. Diff cases']]
daily_report_groupqueue = pd.merge(daily_performance,queue_list,how='left',left_on=['Sampling Queue'],right_on=['QA QUEUE NAME'])
daily_report_groupqueue = daily_report_groupqueue[['Moderation time','MOD QUEUE NAME','No. of Samples','No. Diff cases']]
daily_report_groupqueue = daily_report_groupqueue.groupby(['Moderation time','MOD QUEUE NAME'],as_index=False).agg({'No. of Samples':'sum','No. Diff cases':'sum'})
daily_report_groupqueue['%Accr'] = round(1-(daily_report_groupqueue['No. Diff cases'] / daily_report_groupqueue['No. of Samples']),4)
daily_report_groupqueue

,Moderation time,MOD QUEUE NAME,No. of Samples,No. Diff cases,%Accr
0,2023-07-10,[VN] OEC Product Model Recall Review,28,4.0,0.8571
1,2023-07-11,[VN] OEC Product Model Recall Review,11,2.0,0.8182
2,2023-07-11,[VN] Seller Gov X Feige Chat,18,18.0,0.0000
3,2023-07-12,[VN] OEC Product Model Recall Review,41,1.0,0.9756
4,2023-07-12,[VN] Seller Gov X Feige Chat,10,10.0,0.0000


In [ ]:
all_sheets_gov_oec = {'Diff_case_gov_oec':gov_and_oec_Queue_temp,
                      'Sample_size_gov_oec':sample_size_filter,
                      'False_case_gov_oec':false_case_filter,
                      'Daily_report_group_queue_gov_oec':daily_report_groupqueue}

joblib.dump(all_sheets_gov_oec,'D:/Code/Joblib_Temp/gov_and_oec_Queue_temp.pkl')

['D:/Code/Joblib_Temp/gov_and_oec_Queue_temp.pkl']